In [ ]:
class Competition:
    
    __raise_amount = 1.1
    
    def __init__(self, name, prize):
        self.__name = name
        self.__prize = prize
        
    def get_name(self):
        return self.__name
    def get_prize(self):
        return self.__prize
    def raise_prize(self):
        self.__prize = self.__prize * self.__raise_amount

In [ ]:
race = Competition('race', 100)
type(race)

El método help nos permite nevegar dentro de la clase de forma esquemática. You can pass in a particular class that you've defined, in order to figure out attributes of this class.

- Competition inherits from the built-in object class: Competition(builtins.object)

In [ ]:
help(Competition)

In [ ]:
Competition.__dict__

In [ ]:
class Sprint(Competition):
    pass

In [ ]:
Si hacemos help en Sprint, veremos la class hierarchy
- El metodo hereda de Competition
- El method resolution order(class inheritance hierarchy). El orden será: look for the code for that particular method in Sprint--> Competition --> builins.object

In [ ]:
help(Sprint)

In [ ]:
Sprint.__dict__ # ES un pass, no tiene code defined in its body, it inherits all of the methods and classes

In [ ]:
sprint = Sprint('100m' , 700)
type(sprint)

In [ ]:
sprint.__dict__

In [ ]:
sprint.raise_prize()
sprint.get_prize()

### El método super()

- **super() References the parent of the current child class**. Es un method invokation

-PAsos: 1 Pongo Competition entre paréntesis y luego super().__init__(superclass attributes)

In [ ]:
class Competition:
    
    __raise_amount = 1.1
    
    def __init__(self, name, prize):
        self.__name = name
        self.__prize = prize
        
    def get_name(self):
        return self.__name
    def get_prize(self):
        return self.__prize
    def raise_prize(self):
        self.__prize = self.__prize * self.__raise_amount

In [ ]:
class Cycling(Competition):
    
    def __init__(self, name, prize, country): #NAme and prize are arguments of the __init__ method of the base competition class
        super().__init__(name, prize)
        
        self.__country = country
        
    def get_country(self):
        return self.__country
        

In [ ]:
cycling = Cycling('10km', 7500, 'USA')
cycling.get_country()

In [ ]:
cycling.get_name(), cycling.get_prize()

In [ ]:
class Competition2:
    
    __raise_amount = 1.1
    
    def __init__(self, name):
        self.__name = name

    def get_name(self):
        return self.__name
    def get_prize(self):
        return self.__prize
    def raise_prize(self):
        self.__prize = self.__prize * self.__raise_amount

In [ ]:
class Cycling_2(Competition2):
    
    def __init__(self): #NAme and prize are arguments of the __init__ method of the base competition class
        super().__init__(name = 'Alvaro')
        self.__country = 'España' 
    def get_country(self):
        return self.__country
        

In [ ]:
cycling2 = Cycling_2()
cycling2.get_name(), cycling2.get_country()



**To check whether a particular class is a part of a certain hierarchy** --> issubclass

In [ ]:
issubclass(Cycling, Competition)

In [ ]:
class Shooting(): # IT does not dervie from competition
    
    def __init__(self, name):
        self.first = name

In [ ]:
issubclass(Shooting, Competition)

# Modulo appdata

## ParseTicker

In [1]:
class BaseParser():
    
    def __init__(self, source, target):
        self.__source = source
        self.__target = target
        self.__exceptions ={('Wiki','YahooFinance'):{'IBEX35': '^IBEX','CAC40': '^FCHI','DAX30': '^GDAXI', 'SP500':'^GSPC'},
                            ('Wiki','AlphaVantage'):{}}
        
    
    def get_exceptions(self):
        return self.__exceptions
    
    def set_source(self, source):
        print("Set ticker source in parser")
        self.__source = source
    
    def get_source(self):
        return self.__source
    
    def set_target(self, target):
        print("Set ticker target in parser")
        self.__target = target
    
    def get_target(self):
        return self.__target
       
    def parse(self, name, prefix, suffix, excedent_string):
        key_tuple = (self.__source,self.__target) # Tuple key to select Source-Target exceptions
        
        exception_dict = self.__exceptions.get(key_tuple, {}) #FIX inform not properly loaded. Mirar longitud del dict.

        if name in exception_dict:
            return exception_dict.get(name, name) 
        
        name = name.replace(excedent_string,'')
        return prefix + name + suffix

In [2]:
class WikiToYahooFinance_Parser(BaseParser):
    
    def __init__(self):
        super().__init__(source='Wiki', target='YahooFinance')
        
        self.__parser_keys = {'IBEX35':{'maket':'IBEX35', 'prefix':'','suffix':'.MC','excedent_string':''},
              'CAC40':{'maket':'CAC40','prefix':'','suffix':'.PA','excedent_string':'Euronext: '},
              'DAX30':{'maket':'DAX30','prefix':'','suffix':'.DE', 'excedent_string':''},
              'SP500':{'market':'SP500','prefix':'','suffix':'', 'excedent_string':''}}
    
    def parse_markets(self, raw_ticker): # Feeds tickers for Yahoo finance downloader based on market. Uses parse from BaseParser
        if raw_ticker['market'] == 'IBEX35': #Fix: Repensar esto 
            return super().parse(raw_ticker['ticker'],
                                 self.__parser_keys['IBEX35']['prefix'], 
                                 self.__parser_keys['IBEX35']['suffix'],
                                 self.__parser_keys['IBEX35']['excedent_string'])
        if raw_ticker['market'] == 'CAC40': 
            return super().parse(raw_ticker['ticker'], 
                                 self.__parser_keys['CAC40']['prefix'], 
                                 self.__parser_keys['CAC40']['suffix'],
                                 self.__parser_keys['CAC40']['excedent_string'])
        if raw_ticker['market'] == 'DAX30':
            return  super().parse(raw_ticker['ticker'], 
                                  self.__parser_keys['DAX30']['prefix'], 
                                  self.__parser_keys['DAX30']['suffix'], 
                                  self.__parser_keys['DAX30']['excedent_string'])
        if raw_ticker['market'] == 'SP500':
            return  super().parse(raw_ticker['ticker'], 
                                  self.__parser_keys['SP500']['prefix'], 
                                  self.__parser_keys['SP500']['suffix'],
                                  self.__parser_keys['SP500']['excedent_string'])
        else:
            return raw_ticker['ticker']
        
        
    def feeder_ticker(self,feed_ticker): # Appends feeds to original dictionary, under 'feeds' label. Uses parse_market.
        feed_ticker['feeds'] = {'name':self.get_target(),'ticker':self.parse_markets(feed_ticker)}
        return feed_ticker
    
    
    def parse_all(self, raw_ticker_list): # Loop of feeder_ticker to feed all the tickers in raw_ticker_list
        feed_ticker_list = []
        for raw_ticker in raw_ticker_list:
            feed_ticker_list.append(self.feeder_ticker(raw_ticker))
        print('All tickers fed')
        return feed_ticker_list

## Housekeeping module

In [16]:
import os
from os import path 
import json

class data_housekeeper():
    
    def __init__(self):
        pass

        
    @staticmethod
    def create_dir(myDir):
        if not path.exists(myDir):
            print('Creating dir: ', myDir)
            os.mkdir(myDir)
    
    @staticmethod
    def create_nested_directories(dir_path):
        myPath = ''
        for directory in dir_path:
            myPath += './' + directory
            data_housekeeper.create_dir(myPath)
        #print(myPath)
        
    @staticmethod
    def list_dictionary_to_json(dir_path,file_name, dictionary_list):
        data_housekeeper.create_nested_directories(dir_path)
        file_path = './' + '/'.join(dir_path) +'/'+ file_name
        with open(file_path, 'w') as json_file:
            for ticker_data in dictionary_list:
                #json.dump(ticker, json_file, separators=(":", ",\n"))
                json.dump(ticker_data, json_file, separators=(":", ","))
                json_file.write('\n')
    
    def load_data(self):
        pass

## TickerManager module

In [21]:
from abc import ABC, abstractmethod

class TickerManager(ABC):

    def __init__(self, tickermanager_id, tickers_list):
        #Variables comunes a todos los tipos de tickermanager.
        self.__tickermanager_id = tickermanager_id
        self.__tickers_list = tickers_list # Lista de diccionarios. Ej:
        # [{ "ticker":"BBVA(raw)", "market":"IBEX35", "hora":"YYYY/MM/DD/HH/MM/SS", "source":"Wikipedia", "tickerYahoofinance":"BBVA.MC", "tickerAlphaVantage":"BBVA.BM(inventado)"}]
        self.__update_date = [] #FIX, eliminar, esta variable está incluida en el diccionario
    
    @abstractmethod
    def get_raw_tickers(self):
        pass

    @abstractmethod
    def get_parsed_tickers(self):
        pass
    
    @abstractmethod
    def get_save_tickers(self):
        pass

    def set_tickers_list(self, tickers_list):
        # Sobreescribe la lista tickers_list
        print("Set tickers_list")
        self.__tickers_list = tickers_list
    
    def get_tickers_list(self):
        # Devuelve la lista tickers_list
        print('Getting tickers_list')
        return self.__tickers_list
     
    def get_num_tickers(self):
        # Devuelve el número de diccionarios(uno por cada ticker) en la tickers_list
        print("Number of tickers: {}".format(len(self.__tickers_list)))
        return len(self.__tickers_list)

    def set_tickermanager_id(self, tickermanager_id):
        # Actualiza el par-value del diccionario: source. Ej. "source":"Wikipedia"
        print("Set tickermanager_id: {}".format(tickermanager_id))
        self.__tickermanager_id = tickermanager_id
    
    def get_tickermanager_id(self):
        print("TickerManager id is: {}".format(self.__tickermanager_id))
        return self.__tickermanager_id
    
    def set_update_date(self, update_date):
        # Actualiza el par-value del diccionario: "hora":"YYYY/MM/DD/HH/MM/SS"
        print("Set date of update: {}".format(update_date))
        self.__update_date = update_date #FIX. Actualiza diccionario, no nueva variable

    def get_update_date(self):
        # Devuelve el par-value del diccionario: "hora":"YYYY/MM/DD/HH/MM/SS"
        print("Last date of update id is: {}".format(self.__update_date))
        return self.__update_date
    
    def save_tickers_list(self):
        # Guarda la ticker list. Guarda en JSON.
        dir_path = ['Data']
        file_name = 'tickers.txt'
        data_housekeeper.list_dictionary_to_json(dir_path,file_name, self.get_tickers_list())
        print("Save tickers list")
    

In [34]:
import re
import os
import bs4 as bs
import pandas as pd
import requests
from datetime import date
from datetime import datetime


class TickerManager_Wiki(TickerManager):

    def __init__(self):
        super().__init__(tickermanager_id = "Wiki", tickers_list=[])
        #Variables que sólo sean características del tipo WIKI. Market, url y posiciones en la tabla wiki
        #FIX 
        self.__wiki_metadata = [{'market':'IBEX35','url':'https://es.wikipedia.org/wiki/IBEX_35','pos_table':{'ticker':0, 'company':1,'sector':4,'entry_date':3,'ISIN':5}},
                {'market':'DAX30','url':'https://de.wikipedia.org/wiki/DAX','pos_table':{'ticker':1, 'company':0,'sector':2,'entry_date':5}},
                {'market':'CAC40','url':'https://es.wikipedia.org/wiki/CAC_40','pos_table':{'ticker':2,'company':0, 'sector':1}},
                {'market':'SP500','url':'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies','pos_table':{'ticker':0, 'company':1,'sector':3,'sub_industry':4,'entry_date':6,'CIK':7}}
                ]
        self.__paths = []
        # Instanciar aquí el parser. Para este tickermanger, tener una lista de parsers.
        self.__my_parser_W2YF = WikiToYahooFinance_Parser()
        # El código puede ser genérico, 
    
    #Métodos que sólo son característicos del tipo Wiki
    def get_wikitable_from_url(self,wiki_metadata) -> dict:
        #print(wiki_metadata), print(type(wiki_metadata))
        resp = requests.get(wiki_metadata['url'])
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        table = soup.find('table', {'class': 'wikitable sortable'})
        wikitable_data = []
        for row in table.findAll('tr')[1:]:
            wikitable_dict = {}
            for key in wiki_metadata['pos_table']:
                data = row.findAll('td')[wiki_metadata['pos_table'][key]].text
                wikitable_dict[key] = data.strip()
            wikitable_dict['market'] = wiki_metadata['market']
            wikitable_dict['active_type'] = 'stock'
            wikitable_data.append(wikitable_dict)
        wikitable_data.append({'ticker':wiki_metadata['market'] ,'market':wiki_metadata['market'], 'active_type':'ETF'})
        return wikitable_data
    
    def get_raw_tickers(self):
        # Actualiza los par-value del diccionario: Ticker:value, market:value, name:value.
        # Descargar
        raw_ticker_list = []
        print('Start raw ticker data extraction from Wikipedia ')
        for i,metadata in enumerate(self.__wiki_metadata):
            print('Getting data from market:{}, site:{}'.format(metadata['market'], metadata['url']))
            #print(i), print(metadata)
            wikitable_data = self.get_wikitable_from_url(metadata)
            raw_ticker_list += wikitable_data
        print('Extraction completed')
        super().set_tickers_list(raw_ticker_list)


    #def get_parsed_tickers(self):
    #    # Actualiza los par-value del diccionario: "tickerYahoofinance":"value", "tickerAlphaVantage":"value"
    #    print('Activation: get_parsed_tickers')
    
    def get_parsed_tickers(self): # Función con la que hay que engancahr
        # Actualiza los par-value del diccionario: "tickerYahoofinance":"value", "tickerAlphaVantage":"value"
        print('Activation: get_parsed_tickers')
        feeder_ticker_list = self.__my_parser_W2YF.parse_all(self.get_tickers_list())
        self.set_tickers_list(feeder_ticker_list)
        #return feeder_ticker_list
        
    def get_save_tickers_market(self,market):
        print('Activation: get_save_tickers_market. Getting tickers with source in Wikipedia. Market: {}'.format(market))
        pass
        # Función alto nivel de la clase TickerManager_Wiki. 
        # Obtiene una lista de diccionarios, un diccionario por ticker con origen Wikipedia para el mercado especificado y lo guarda.
    
    def get_save_tickers(self):
        print('Get tickers with source in Wikipedia. All markets')
        pass
        # Función alto nivel de la clase TickerManager_Wiki. 
        # Obtiene una lista de diccionarios, un diccionario por ticker con origen Wiki, para cada uno de los mercados que existen y lo guarda.
        # Incluye los siguientes métodos

In [23]:
tickerMan = TickerManager_Wiki()
tickerMan.get_tickermanager_id()
tickerMan.set_update_date('1981')
tickerMan.get_update_date()
tickerMan.set_tickermanager_id('Wiki2')
tickerMan.get_tickermanager_id()
tickerMan.get_num_tickers()
tickerMan.get_num_tickers()
tickerMan.get_save_tickers_market('IBEX35')
tickerMan.get_save_tickers()
tickerMan.get_raw_tickers()
tickerMan.get_parsed_tickers()
tickerMan.save_tickers_list()

TickerManager id is: Wiki
Set date of update: 1981
Last date of update id is: 1981
Set tickermanager_id: Wiki2
TickerManager id is: Wiki2
Number of tickers: 1
Number of tickers: 1
Activation: get_save_tickers_market. Getting tickers with source in Wikipedia. Market: IBEX35
Get tickers with source in Wikipedia. All markets
Start raw ticker data extraction from Wikipedia 
Getting data from market:IBEX35, site:https://es.wikipedia.org/wiki/IBEX_35
Getting data from market:DAX30, site:https://de.wikipedia.org/wiki/DAX
Getting data from market:CAC40, site:https://es.wikipedia.org/wiki/CAC_40
Getting data from market:SP500, site:http://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Extraction completed
Set tickers_list
Activation: get_parsed_tickers
Getting tickers_list
All tickers fed
Set tickers_list
Getting tickers_list
Save tickers list


In [35]:
tickerMan = TickerManager_Wiki()
tickerMan.get_parsed_tickers()

Activation: get_parsed_tickers
Getting tickers_list
All tickers fed
Set tickers_list


In [36]:
tickerMan.get_tickers_list()

Getting tickers_list


[]

In [37]:
if not tickerMan.get_tickers_list():
    print('empty')

Getting tickers_list
empty


In [15]:
list1 = [{"Alvaro":"1.72"},
         {"Ricardo": "1,79"},
         {"Ferri":"1,7"}
        ]
len(list1)

3

In [ ]:
class TickerManagerFactory:
    def __init__(self) -> object:
        pass
    
    @staticmethod
    def get_names() -> list:
        tickermanager_names = []
        tickermanager_names.append('TickerManager_Wiki')
        return tickermanager_names
    
    @staticmethod
    def create(tickermanager_name, *args) -> TickerManager :
        if tickermanager_name == 'TickerManager_Wiki':
            return TickerManager_Wiki()

        else:
            # TODO raise exception
            print('Create TickerManager: Unknown Tickermanager {}'.format(tickermanager_name))
            return None
    
    
    

In [ ]:
tickerMan2 = TickerManagerFactory.create('TickerManager_Wiki')

In [ ]:
tickerMan2 = TickerManager_Wiki()
tickerMan2.get_tickermanager_id()
tickerMan2.set_update_date('1981')
tickerMan2.get_update_date()
tickerMan2.set_tickermanager_id('Wiki2')
tickerMan2.get_tickermanager_id()
tickerMan2.get_raw_tickers()
tickerMan2.get_num_tickers()
tickerMan2.get_num_tickers()
tickerMan2.get_parsed_tickers()
tickerMan2.save_tickers_list()
tickerMan2.get_save_tickers_market('IBEX35')
tickerMan2.get_save_tickers()